In [ ]:
class DoubleConv(nn.Module):
    def __init__(self,in_ch,out_ch):
        super(DoubleConv,self).__init__()
        self.conv = nn.Sequential(
                nn.Conv2d(in_ch,out_ch,3,padding=1),#in_ch、out_ch是通道数
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace = True),
                nn.Conv2d(out_ch,out_ch,3,padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace = True)  
            )
    def forward(self,x):
        return self.conv(x)

conv1 = DoubleConv(3,64)
pool1 = nn.MaxPool2d(2)
conv2 = DoubleConv(64,128)
pool2 = nn.MaxPool2d(2)
conv3 = DoubleConv(128,256)
pool3 = nn.MaxPool2d(2)
conv4 = DoubleConv(256,512)
pool4 = nn.MaxPool2d(2)
conv5 = DoubleConv(512,1024)
up6 = nn.ConvTranspose2d(1024,512,2,stride=2)
conv6 = DoubleConv(1024,512)
up7 = nn.ConvTranspose2d(512,256,2,stride=2)
conv7 = DoubleConv(512,256)
up8 = nn.ConvTranspose2d(256,128,2,stride=2)
conv8 = DoubleConv(256,128)
up9 = nn.ConvTranspose2d(128,64,2,stride=2)
conv9 = DoubleConv(128,64)
out_ch = 1
conv10 = nn.Conv2d(64,out_ch,1)

In [ ]:
from tqdm import tqdm

liver_dataset = LiverDataset()
dataloader = DataLoader(liver_dataset, 4, shuffle=True,num_workers=1)
for __num__,(x, y) in enumerate(tqdm(dataloader)):
    inputs = x.to(device, dtype = torch.float)
    labels = y.to(device, dtype = torch.float)
    labels = labels.unsqueeze(1)
    c1 = conv1(inputs)
    p1 = pool1(c1)
    c2 = conv2(p1)
    p2 = pool2(c2)
    c3 = conv3(p2)
    p3 = pool3(c3)
    c4 = conv4(p3)
    p4 = pool4(c4)
    c5 = conv5(p4)
    up_6 = up6(c5)
    merge6 = torch.cat([up_6,c4],dim=1)#按维数1（列）拼接,列增加
    c6 = conv6(merge6)
    up_7 = up7(c6)
    merge7 = torch.cat([up_7,c3],dim=1)
    c7 = conv7(merge7)
    up_8 = up8(c7)
    merge8 = torch.cat([up_8,c2],dim=1)
    c8 = conv8(merge8)
    up_9 = up9(c8)
    merge9 = torch.cat([up_9,c1],dim=1)
    c9 = conv9(merge9)
    c10 = conv10(c9)
    out = nn.Sigmoid()(c10)#化成(0~1)区间
    if __num__ == 0:
        break
print(c1.shape)
print(p1.shape)
print(c2.shape)
print(p2.shape)
print(c3.shape)
print(p3.shape)
print(c4.shape)
print(p4.shape)
print(c5.shape)
print(up_6.shape)

labels[0][0].shape

In [ ]:
data_dir_clip = './data/aisegmentcom-matting-human-datasets/clip_img/1803151818/clip_00000000'
#         data_dir_clip = '/home/vivi/sace/Google_ML_Camp/bk_rm/data/clip_img/1803151818/clip_00000000'

data_dir_mat = './data/aisegmentcom-matting-human-datasets/matting/1803151818/matting_00000000'
#         data_dir_mat = '/home/vivi/sace/Google_ML_Camp/bk_rm/data/matting/1803151818/clip_00000000'
    
        # n = len(os.listdir(data_dir_clip))
clip_imgs = os.listdir(data_dir_clip)
        # mat_imgs = os.listdir(data_dir_mat)
imgs = []

for enu_num, img_clip in enumerate(clip_imgs):
    img_mat = img_clip[0:-4]+'.png'
    img_clip_path = data_dir_clip + '/' + img_clip
    img_mat_path = data_dir_mat + '/' + img_mat

    imgs.append([img_clip_path,img_mat_path])

    if enu_num == 0:
        break

img_y_ori = cv2.imread(img_mat_path, cv2.IMREAD_UNCHANGED)
img_y = img_y_ori[:,:,3]
img_y = img_y[144:656, 44:556]

In [ ]:
testset = (img_y>1)*1
testset